##import

### import

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pycaret.regression import *
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import joblib
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from pycaret import *
import optuna

### 메소드

In [2]:

# smape 계산 함수
def smape(true, pred):
  v = 2*abs(pred-true)/(abs(pred)+abs(true))
  output=np.mean(v)*100
  return output

# custom metrics
def custom_metrics(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

### 데이터 불러오기

In [3]:
path_drive = './csv/'
path = './'

In [4]:
combined_clust_df = pd.read_csv(path_drive+'combined_clust_df.csv')
test_df = pd.read_csv(path_drive + 'test_df.csv')

In [5]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

train_cdhs = np.array([])
test_cdhs = np.array([])

for num in range(1,101,1):
    train_temp = combined_clust_df[combined_clust_df['건물번호'] == num]
    test_temp = test_df[test_df['건물번호'] == num]
    train_cdh = CDH(train_temp['기온(C)'].values)
    test_cdh = CDH(test_temp['기온(C)'].values)
    train_cdhs = np.concatenate([train_cdhs, train_cdh])
    test_cdhs = np.concatenate([test_cdhs, test_cdh])
combined_clust_df['CDH'] = train_cdhs
test_df['CDH'] = test_cdhs
# combined_clust_df.head(30)

In [209]:
# combined_clust_df.reset_index(inplace=True)
# test_df.reset_index(inplace=True)
# tmp_grouped = combined_clust_df.groupby(['건물번호','시간','요일'])['전력소비량(kWh)'].mean().reset_index()
# tmp_grouped.rename(columns={'전력소비량(kWh)':'요일_시간_mean'}, inplace=True)
# combined_clust_df = combined_clust_df.merge(tmp_grouped, on=['건물번호','시간','요일'])
# test_df = test_df.merge(tmp_grouped, on=['건물번호','시간','요일'])

# tmp_grouped = combined_clust_df.groupby(['건물번호','시간'])['전력소비량(kWh)'].mean().reset_index()
# tmp_grouped.rename(columns={'전력소비량(kWh)':'시간_mean'}, inplace=True)
# combined_clust_df = combined_clust_df.merge(tmp_grouped, on=['건물번호','시간'])
# test_df = test_df.merge(tmp_grouped, on=['건물번호','시간'])

# tmp_grouped = combined_clust_df.groupby(['건물번호','시간'])['전력소비량(kWh)'].std().reset_index()
# tmp_grouped.rename(columns={'전력소비량(kWh)':'시간_std'}, inplace=True)
# combined_clust_df = combined_clust_df.merge(tmp_grouped, on=['건물번호','시간'])
# test_df = test_df.merge(tmp_grouped, on=['건물번호','시간'])
# combined_clust_df = combined_clust_df.sort_values(by=['index'], axis=0)
# test_df = test_df.sort_values(by=['index'], axis=0)
# combined_clust_df=combined_clust_df.drop('index',axis=1)
# test_df=test_df.drop('index',axis=1)

In [90]:
test_df

,건물번호,일시,기온(C),풍속(m/s),습도(%),불쾌지수,불쾌지수_이동평균3,불쾌지수_이동평균5,holiday,월,...,할인마트,호텔및리조트,건물유형_라벨,태양광여부,work_time,low_day,particular,요일_시간_mean,시간_mean,시간_std
0,1,2022-08-25 00:00:00,23.5,2.2,72,71.78980,71.789800,71.789800,False,8,...,0,0,1,False,0,0,0,1627.800000,1706.318118,446.882767
7,1,2022-08-25 01:00:00,23.0,0.9,72,71.02840,71.409100,71.409100,False,8,...,0,0,1,False,0,0,0,1550.080000,1622.620235,439.662704
14,1,2022-08-25 02:00:00,22.7,1.5,75,70.81675,71.211650,71.211650,False,8,...,0,0,1,False,0,0,0,1431.120000,1506.971294,412.071906
21,1,2022-08-25 03:00:00,22.1,1.3,78,70.11262,70.652590,70.936892,False,8,...,0,0,1,False,0,0,0,1372.200000,1437.365647,391.205981
28,1,2022-08-25 04:00:00,21.8,1.0,77,69.56514,70.164837,70.662542,False,8,...,0,0,1,False,0,0,0,1381.720000,1447.321412,381.099697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16771,100,2022-08-31 19:00:00,22.5,0.9,84,71.22400,72.378973,72.792244,False,8,...,0,1,12,False,1,0,0,964.873846,1010.462118,161.399578
16778,100,2022-08-31 20:00:00,20.7,0.4,95,68.95035,70.961363,71.899884,False,8,...,0,1,12,False,1,0,0,882.184615,928.125176,137.566008
16785,100,2022-08-31 21:00:00,20.2,0.4,98,68.24604,69.473463,70.866662,False,8,...,0,1,12,False,1,0,0,779.095385,830.032941,128.300189
16792,100,2022-08-31 22:00:00,20.1,1.1,97,68.01203,68.402807,69.828432,False,8,...,0,1,12,False,1,0,0,663.267692,723.100235,112.464079


## 군집화

### 각 군집별로 모델 생성(et) +모델별 feature 추가

In [15]:
combined_clust_df.columns.tolist()

['건물번호',
 '일시',
 '기온(C)',
 '풍속(m/s)',
 '습도(%)',
 '전력소비량(kWh)',
 '불쾌지수',
 '불쾌지수_이동평균3',
 '불쾌지수_이동평균5',
 'holiday',
 '월',
 '일',
 '시간',
 '시간_sin',
 '시간_cos',
 '요일',
 '건물유형',
 '연면적(m2)',
 '냉방면적(m2)',
 '태양광용량(kW)',
 'ESS저장용량(kWh)',
 '건물기타',
 '공공',
 '대학교',
 '데이터센터',
 '백화점및아울렛',
 '병원',
 '상용',
 '아파트',
 '연구소',
 '지식산업센터',
 '할인마트',
 '호텔및리조트',
 '건물유형_라벨',
 '태양광여부',
 'km_cluster',
 'datetime',
 'work_time',
 'low_day',
 'particular',
 'CDH']

Cluster 0: ['건물기타', '대학교', '연구소', '할인마트']  
Cluster 1: ['백화점및아울렛', '호텔및리조트']  
Cluster 2: ['데이터센터', '아파트']  
Cluster 3: ['공공', '병원', '상용', '지식산업센터']

In [7]:
clust_to_num = {0: [1, 3, 9, 11], 1: [5, 12], 2: [4, 8], 3: [2, 6, 7, 10]}

In [8]:
model_type = 'et'

# et, rf, catboost, xgboost, lightgbm, dt 6개 앙상블 해보기 (train 기준 smape 찍고 상위 몇가지 알고리즘만 채택)

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.ensemble_model


# clust0

In [32]:
selected_columns0 = ['전력소비량(kWh)',
 '연면적(m2)',
  '연구소',
  'work_time',
  '건물번호',
  'holiday',
  'particular',
  '냉방면적(m2)',
  '시간_cos',
  '대학교',
  '불쾌지수',
  'low_day',
  '불쾌지수_이동평균3',
  '시간',
  '태양광용량(kW)',
  'ESS저장용량(kWh)',
  '월',
  '시간_sin',
  '기온(C)',
  '요일',
  '일', 'CDH',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns0]

clust0_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 0].reset_index(drop=True)

# con_li = ['연면적(m2)','냉방면적(m2)','시간_cos','불쾌지수',
#           '불쾌지수_이동평균3','시간','태양광용량(kW)','ESS저장용량(kWh)',
#           '시간_sin','기온(C)','CDH']

# sc = StandardScaler()
# clust0_train_df[con_li] = sc.fit_transform(clust0_train_df[con_li])

# clust0_train_df['전력소비량(kWh)'] = clust0_train_df['전력소비량(kWh)'].map(lambda x:np.log(x) if x > 0 else 0)
# 로그변환 오히려 성능 저하

In [39]:
cat_li = ['건물번호',  'holiday',  'particular','low_day']

# pycaret 설정
exp0 = setup(data=clust0_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)', 
             categorical_features=cat_li, train_size=len(clust0_train_df)-clust0_train_df['건물번호'].nunique()*168)
exp0.add_metric('smape','SMAPE',smape,greater_is_better=False)

# et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')

,Description,Value
0,Session id,8573
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(79560, 22)"
4,Transformed data shape,"(79560, 22)"
5,Transformed train set shape,"(73008, 22)"
6,Transformed test set shape,"(6552, 22)"
7,Ordinal features,3
8,Numeric features,17
9,Categorical features,4


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,87.3738,23516.0679,153.3495,0.9975,0.0715,0.0418,4.0855
1,86.9186,23214.5676,152.3633,0.9977,0.0727,0.0423,4.1192
2,85.9051,22464.0743,149.8802,0.9978,0.0713,0.0421,4.1025
3,89.8583,25658.3801,160.1823,0.9976,0.0793,0.0445,4.2522
4,84.3516,20951.1184,144.7450,0.9978,0.0705,0.0416,4.0339
5,86.6626,25891.3875,160.9080,0.9976,0.0715,0.0418,4.0754
6,86.4341,24673.3238,157.0774,0.9975,0.0737,0.0423,4.0949
7,86.0145,24313.9494,155.9293,0.9974,0.0765,0.0428,4.1239
8,85.4887,22150.1743,148.8293,0.9978,0.0722,0.0419,4.0400


In [13]:
from pycaret.regression import *
from sklearn.ensemble import ExtraTreesRegressor

X_train, X_valid, y_train, y_valid = train_test_split(clust0_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1), clust0_train_df['전력소비량(kWh)'], test_size=168*clust0_train_df['건물번호'].nunique())
def objective(trial):
    param = {
        'n_estimators': 1000,
    'max_depth' : trial.suggest_int('max_depth', 20, 30),
    'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
 'max_features': 1.0,
            'criterion': 'squared_error',
 'random_state': 5568,
        'n_jobs':-1,
 'verbose': 1
    }

    model = ExtraTreesRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(y_valid, preds)

    return sm


study = optuna.create_study(study_name = 'ET Optuna_cluster_0')
study.optimize(objective, n_trials=30, timeout=24000)

et = study.best_trial
et_param = et.params
print('Best Trial : score {},\nparams {}'.format(et.value, et_param))

[I 2023-08-28 06:56:16,608] A new study created in memory with name: ET Optuna_cluster_0
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 760 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.6s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 760 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 1000 out of 1000 | elapsed:    0.3s finished
[I 2023-08-28 06:56:24,879] Trial 0 finished with value: 4.294492143785948 and parameters: {'max_depth': 

Best Trial : score 3.8013519583829014,
params {'max_depth': 30, 'min_samples_split': 5, 'min_samples_leaf': 1}


In [23]:
from sklearn.ensemble import RandomForestRegressor
X_train, X_valid, y_train, y_valid = train_test_split(clust0_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1), clust0_train_df['전력소비량(kWh)'], test_size=168*clust0_train_df['건물번호'].nunique())

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 10, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 5),
        'random_state': 4555,
 'n_jobs': -1,
 'verbose': 1
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(y_valid, preds)

    
    return sm

study_0 = optuna.create_study(study_name = 'RF Optuna_0')
study_0.optimize(objective, n_trials=50, timeout=24000)

rf_0 = study_0.best_trial
rf_0_param = rf_0.params
print('Best Trial: score {},\nparams {}'.format(rf_0.value, rf_0_param))

[I 2023-08-28 07:14:59,345] A new study created in memory with name: RF Optuna_0
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 679 out of 679 | elapsed:   14.1s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 679 out of 679 | elapsed:    0.2s finished
[I 2023-08-28 07:15:13,755] Trial 0 finished with value: 4.280863897155934 and parameters: {'n_estimators': 679, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 0 with value: 4.280863897155934.
[Parallel(n_jobs

Best Trial: score 4.113664005914089,
params {'n_estimators': 925, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 3}


In [196]:
# 앙상블


cat_li = ['건물번호',  'holiday',  'particular','low_day']

# pycaret 설정
exp0 = setup(data=clust0_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)', 
             categorical_features=cat_li, train_size=len(clust0_train_df)-clust0_train_df['건물번호'].nunique()*168)
exp0.add_metric('smape','SMAPE',smape,greater_is_better=False)
# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
# catboost_model = create_model('catboost')  # catboost 모델
# lightgbm_model = create_model('lightgbm')  # LightGBM 모델
# dt_model = create_model('dt')  # dt 모델
# xgboost_model = create_model('xgboost')  # xgboost 모델

,Description,Value
0,Session id,6112
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(79560, 22)"
4,Transformed data shape,"(79560, 22)"
5,Transformed train set shape,"(73008, 22)"
6,Transformed test set shape,"(6552, 22)"
7,Ordinal features,3
8,Numeric features,17
9,Categorical features,4


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,78.9182,19877.0623,140.9860,0.9981,0.0755,0.0408,3.9287
1,79.7442,20946.3914,144.7287,0.9979,0.0754,0.0409,3.9305
2,77.2876,18055.9141,134.3723,0.9982,0.0742,0.0399,3.8451
3,79.0941,20150.5588,141.9527,0.9981,0.0765,0.0404,3.8540
4,76.1472,19415.0085,139.3377,0.9981,0.0722,0.0384,3.7025
5,76.4158,19506.6979,139.6664,0.9979,0.0734,0.0393,3.7652
6,76.2784,18440.1599,135.7946,0.9981,0.0727,0.0391,3.7569
7,76.1263,18360.7427,135.5018,0.9981,0.0716,0.0392,3.8143
8,76.8210,19114.0577,138.2536,0.9983,0.0739,0.0394,3.8122


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,86.7580,22458.3063,149.8610,0.9978,0.0725,0.0428,4.1624
1,86.8145,24275.2431,155.8051,0.9976,0.0723,0.0423,4.0814
2,86.4839,23308.8960,152.6725,0.9977,0.0743,0.0423,4.1000
3,87.0402,23080.8463,151.9238,0.9978,0.0737,0.0420,4.0546
4,86.1432,26686.8727,163.3612,0.9974,0.0715,0.0409,3.9750
5,86.8125,25898.0940,160.9288,0.9972,0.0739,0.0423,4.0929
6,83.5343,21639.0354,147.1021,0.9978,0.0713,0.0410,3.9684
7,85.7959,22348.9859,149.4958,0.9977,0.0713,0.0421,4.1108
8,87.0140,23555.7857,153.4789,0.9979,0.0720,0.0421,4.1001


In [170]:
m_0 = [et_model, rf_model]
w_li = [[0.7,0.3],[0.75,0.25],[0.8,0.2],[0.9,0.1]]
b_li = []
for w in w_li:
    blender = blend_models(m_0, weights=w)
    b_li.append(blender)

# save_model(blender,'./ensemble-0')

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,78.2930,19776.0690,140.6274,0.9981,0.0739,0.0402,3.8470
1,76.3965,18638.4359,136.5227,0.9981,0.0694,0.0384,3.7179
2,76.8714,18005.3061,134.1839,0.9981,0.0685,0.0385,3.7696
3,77.8874,18901.4317,137.4825,0.9982,0.0718,0.0393,3.7953
4,79.7565,20723.5260,143.9567,0.9979,0.0706,0.0394,3.8309
5,79.6220,19550.2117,139.8221,0.9980,0.0737,0.0406,3.9155
6,78.3124,19225.7232,138.6569,0.9980,0.0715,0.0394,3.7964
7,75.9520,18044.0971,134.3283,0.9983,0.0706,0.0390,3.7603
8,75.4925,17654.4113,132.8699,0.9981,0.0769,0.0406,3.8464


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,78.0903,19687.4536,140.3120,0.9981,0.0741,0.0402,3.8459
1,76.2198,18575.9947,136.2938,0.9981,0.0696,0.0384,3.7147
2,76.6103,17870.9212,133.6822,0.9982,0.0687,0.0384,3.7669
3,77.6408,18830.4822,137.2242,0.9982,0.0720,0.0392,3.7911
4,79.5627,20689.7562,143.8393,0.9979,0.0710,0.0395,3.8290
5,79.3864,19437.8502,139.4197,0.9980,0.0739,0.0406,3.9120
6,78.1058,19187.1991,138.5179,0.9980,0.0716,0.0394,3.7908
7,75.6759,17961.9215,134.0221,0.9983,0.0708,0.0390,3.7546
8,75.3139,17647.0617,132.8422,0.9981,0.0774,0.0406,3.8458


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,77.9336,19635.2509,140.1258,0.9981,0.0744,0.0402,3.8471
1,76.1016,18540.8408,136.1648,0.9981,0.0699,0.0384,3.7142
2,76.4249,17776.5736,133.3288,0.9982,0.0690,0.0385,3.7673
3,77.4534,18793.6122,137.0898,0.9983,0.0722,0.0392,3.7889
4,79.4266,20688.3411,143.8344,0.9979,0.0715,0.0395,3.8295
5,79.1982,19359.0236,139.1367,0.9981,0.0742,0.0406,3.9110
6,77.9509,19177.9681,138.4845,0.9980,0.0719,0.0393,3.7877
7,75.4782,17914.1416,133.8437,0.9983,0.0710,0.0390,3.7519
8,75.1884,17667.5671,132.9194,0.9981,0.0779,0.0407,3.8474


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,77.7502,19640.0835,140.1431,0.9981,0.0750,0.0403,3.8550
1,76.0359,18552.3949,136.2072,0.9981,0.0705,0.0384,3.7207
2,76.2678,17707.9902,133.0714,0.9982,0.0697,0.0386,3.7772
3,77.2501,18822.1104,137.1937,0.9982,0.0728,0.0393,3.7906
4,79.3116,20782.5753,144.1616,0.9979,0.0725,0.0397,3.8380
5,78.9755,19301.9758,138.9316,0.9981,0.0749,0.0406,3.9156
6,77.8358,19247.3852,138.7349,0.9980,0.0725,0.0394,3.7893
7,75.2949,17921.7690,133.8722,0.9983,0.0716,0.0390,3.7537
8,75.1359,17792.1431,133.3872,0.9981,0.0790,0.0409,3.8580


In [24]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 4555,
 'verbose': 0,
 'warm_start': False}

In [77]:
et_params_0= {'n_estimators':1000, 'max_depth': 29, 'min_samples_split': 4, 'min_samples_leaf': 1,
       'max_features': 1.0,'criterion': 'squared_error', 'random_state': 5568}
rf_params_0 = {'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 4555,
 'verbose': 0,
 'warm_start': False}
et_0 = ExtraTreesRegressor(**et_params_0)
rf_0 = RandomForestRegressor(**rf_params_0)

In [40]:
m_0 = [et_0, rf_model]
blender = blend_models(m_0, weights=[0.8,0.2])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,78.3715,18496.2225,136.0008,0.9980,0.0724,0.0396,3.8334
1,79.2102,19419.7327,139.3547,0.9981,0.0723,0.0400,3.8536
2,76.6381,17836.7862,133.5544,0.9982,0.0697,0.0390,3.7947
3,81.2216,21411.3483,146.3262,0.9980,0.0794,0.0420,3.9764
4,76.6033,17588.6534,132.6222,0.9982,0.0701,0.0393,3.7974
5,78.1786,19993.8871,141.3997,0.9982,0.0692,0.0390,3.7914
6,77.2788,19061.1234,138.0620,0.9980,0.0732,0.0400,3.8343
7,78.1367,18850.2718,137.2963,0.9980,0.0744,0.0403,3.8686
8,77.0118,17789.7384,133.3782,0.9982,0.0717,0.0396,3.7927


In [41]:
blender_1 = blend_models(m_0, weights=[0.9,0.1])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,78.1013,18424.8829,135.7383,0.9980,0.0732,0.0397,3.8386
1,79.0852,19489.1961,139.6037,0.9981,0.0731,0.0400,3.8549
2,76.2877,17735.5262,133.1748,0.9982,0.0701,0.0390,3.7916
3,81.0340,21395.8278,146.2731,0.9980,0.0801,0.0421,3.9778
4,76.5284,17658.3996,132.8849,0.9981,0.0706,0.0393,3.8029
5,77.9960,19827.9011,140.8116,0.9982,0.0697,0.0391,3.7937
6,76.9644,18912.6033,137.5231,0.9981,0.0738,0.0400,3.8364
7,77.8509,18671.5715,136.6440,0.9980,0.0750,0.0403,3.8675
8,76.7632,17787.9436,133.3714,0.9982,0.0723,0.0396,3.7944


In [78]:
# f_blender = finalize_model(blender)
save_model(et_0,'./ensemble-0')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['CDH', 'work_time', '연면적(m2)',
                                              '냉방면적(m2)', '시간', '시간_cos', '불쾌지수',
                                              '월', '기온(C)', '시간_sin',
                                              '불쾌지수_이동평균3', '요일', '일', '습도(%)'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['건물번호', 'holiday'],
                                     transformer=SimpleImputer(strategy...
 NaN     -1
 dtype: int64}]))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['건물번호'],
                                     transformer=OneHotEncoder(cols=['건물번호'],
                                                               handle_missing='return_nan',
                                                        

In [226]:
et_model = finalize_model(et_model)
rf_model = finalize_model(rf_model)
m_0 = [et_model, rf_model]
blender = blend_models(m_0, weights=[0.8,0.2])
save_model(blender,'./ensemble-0')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,04:11:55
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Voting Regressor


KeyboardInterrupt: 

In [66]:
m_2 = [et_model_final, rf_model_final, dt_model_final]
blender = blend_models(m_2)

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,85.2111,23463.0599,153.1766,0.9976,0.0725,0.0412,3.9953
1,83.6376,22198.5447,148.9918,0.9980,0.0780,0.0423,4.0609
2,83.4902,21909.2094,148.0176,0.9977,0.0762,0.0422,4.0581
3,84.7209,23789.6692,154.2390,0.9975,0.0761,0.0427,4.1366
4,80.0577,20367.0005,142.7130,0.9979,0.0710,0.0401,3.8824
5,82.6235,23560.5397,153.4944,0.9976,0.0701,0.0393,3.8550
6,81.1861,20341.4852,142.6236,0.9981,0.0704,0.0401,3.9183
7,83.9473,23567.1286,153.5159,0.9977,0.0734,0.0411,3.9637
8,85.9180,24600.3760,156.8451,0.9975,0.0708,0.0406,3.9657


# clust1

In [64]:
selected_columns1 = ['전력소비량(kWh)','CDH',
'work_time', '연면적(m2)', '냉방면적(m2)', '시간', '건물번호', '시간_cos', 
                     '불쾌지수', '월', '기온(C)', 'holiday', '시간_sin', '불쾌지수_이동평균3',
                     '요일', '일', '습도(%)', 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns1]

clust1_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 1].reset_index(drop=True)

# con_li = ['연면적(m2)','냉방면적(m2)','시간_cos','CDH',
#           '불쾌지수','불쾌지수_이동평균3','시간',
#           '시간_sin','기온(C)','습도(%)']

# sc = StandardScaler()
# clust1_train_df[con_li] = sc.fit_transform(clust1_train_df[con_li])


In [65]:
# 앙상블
cat_li = ['건물번호',  'holiday']

# pycaret 설정
exp0 = setup(data=clust1_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)', 
             categorical_features=cat_li, train_size=len(clust1_train_df)-clust1_train_df['건물번호'].nunique()*168)
exp0.add_metric('smape','SMAPE',smape,greater_is_better=False)
# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
rf_model = create_model('rf')  # 랜덤 포레스트 모델
# catboost_model = create_model('catboost')  # catboost 모델
# lightgbm_model = create_model('lightgbm')  # LightGBM 모델
# dt_model = create_model('dt')  # dt 모델
# xgboost_model = create_model('xgboost')

,Description,Value
0,Session id,6450
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(32640, 17)"
4,Transformed data shape,"(32640, 32)"
5,Transformed train set shape,"(29952, 32)"
6,Transformed test set shape,"(2688, 32)"
7,Ordinal features,1
8,Numeric features,14
9,Categorical features,2


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,69.7745,18497.2417,136.0046,0.9899,0.0921,0.0513,4.8534
1,71.7950,22422.0598,149.7400,0.9888,0.0887,0.0500,4.7688
2,69.2646,17799.3739,133.4143,0.9902,0.0915,0.0511,4.8290
3,67.4954,15066.8984,122.7473,0.9919,0.0886,0.0500,4.8254
4,66.0295,15271.4915,123.5779,0.9915,0.0789,0.0468,4.5611
5,71.4795,16994.6370,130.3635,0.9913,0.0870,0.0500,4.8187
6,66.7756,16606.2598,128.8653,0.9912,0.0896,0.0498,4.7017
7,68.3327,16631.1239,128.9617,0.9911,0.0878,0.0500,4.7816
8,66.5427,15724.6646,125.3980,0.9913,0.1486,0.0490,4.7651


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,75.6011,22588.2101,150.2937,0.9877,0.1001,0.0557,5.1919
1,78.7033,24779.2449,157.4142,0.9876,0.0940,0.0545,5.1621
2,78.0707,22760.9818,150.8674,0.9874,0.0978,0.0559,5.2466
3,77.8568,23197.1429,152.3061,0.9876,0.1007,0.0566,5.3422
4,71.8981,17738.1035,133.1845,0.9901,0.0824,0.0498,4.8809
5,77.5958,20250.3980,142.3039,0.9896,0.0949,0.0546,5.1788
6,75.1074,23263.1053,152.5225,0.9877,0.0996,0.0555,5.1271
7,74.2846,19461.8392,139.5057,0.9896,0.0949,0.0546,5.1633
8,74.4017,19165.1543,138.4383,0.9894,0.1482,0.0535,5.1619


In [35]:
exp = setup(data=clust1_train_df, target='전력소비량(kWh)')
exp.add_metric('smape','SMAPE',smape,greater_is_better=False)

# 모델 학습 및 비교
best_model = compare_models(sort='SMAPE')
print(best_model)

,Description,Value
0,Session id,5255
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(32640, 18)"
4,Transformed data shape,"(32640, 18)"
5,Transformed train set shape,"(22848, 18)"
6,Transformed test set shape,"(9792, 18)"
7,Numeric features,16
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE,TT (Sec)
et,Extra Trees Regressor,74.4341,21481.2060,146.1484,0.9885,0.1090,0.1365,5.1345,0.8010
rf,Random Forest Regressor,81.5179,25469.7488,158.9964,0.9864,0.1160,0.1587,5.4986,1.1960
catboost,CatBoost Regressor,96.0820,26022.7761,161.1734,0.9861,0.1248,0.1890,6.7048,3.0840
xgboost,Extreme Gradient Boosting,100.8020,28648.7504,169.1114,0.9847,0.1282,0.1644,6.9534,0.2650
dt,Decision Tree Regressor,104.0740,46776.9889,214.8390,0.9750,0.1449,0.0736,7.0221,0.0350
lightgbm,Light Gradient Boosting Machine,108.1575,32175.5942,179.1481,0.9828,0.1333,0.2198,7.4647,0.0680
gbr,Gradient Boosting Regressor,199.4640,94531.2340,307.2620,0.9495,0.2218,0.3218,13.9553,0.3540
knn,K Neighbors Regressor,483.9133,473313.6531,687.7904,0.7470,0.3802,0.4872,27.2570,0.0310
ada,AdaBoost Regressor,632.5651,501829.9636,708.2379,0.7317,0.5284,0.7644,41.1191,0.4380
en,Elastic Net,771.4350,916573.8062,957.3254,0.5101,0.6736,0.9615,48.4555,0.0190


ExtraTreesRegressor(n_jobs=-1, random_state=5255)


In [134]:
m_0 = [et_model, rf_model]
blender_0 = blend_models(m_0, weights=[0.5,0.5])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,69.7821,17380.1415,131.8338,0.9910,0.0910,0.0504,4.6816
1,67.6826,15712.3126,125.3488,0.9913,0.1487,0.0489,4.8142
2,75.8111,23199.2871,152.3131,0.9876,0.0976,0.0541,5.1529
3,75.2306,21367.2760,146.1755,0.9889,0.0976,0.0557,5.2220
4,70.3739,17791.8429,133.3861,0.9903,0.0823,0.0482,4.6623
5,68.5260,13774.4228,117.3645,0.9924,0.0836,0.0498,4.8895
6,75.0048,29739.5043,172.4515,0.9839,0.1086,0.0565,5.0435
7,71.6872,21264.0467,145.8220,0.9888,0.1486,0.7149,4.9964
8,73.4319,20755.7708,144.0686,0.9888,0.0908,0.0516,4.9214


In [135]:
w_li = [[0.6,0.4],[0.65,0.35],[0.7,0.3],[0.75,0.25],[0.8,0.2],[0.85,0.15]]
b_li = []
for w in w_li:
    blender = blend_models(m_0, weights=w)
    b_li.append(blender)

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,69.1027,16713.8544,129.2821,0.9913,0.0900,0.0499,4.6467
1,67.2579,15684.8979,125.2394,0.9913,0.1486,0.0486,4.7890
2,75.0921,22529.0767,150.0969,0.9880,0.0967,0.0536,5.1188
3,74.3251,20861.8257,144.4362,0.9891,0.0967,0.0551,5.1725
4,69.7723,17684.5045,132.9831,0.9903,0.0820,0.0478,4.6259
5,67.8701,13473.4415,116.0752,0.9926,0.0832,0.0495,4.8566
6,74.1154,29035.8909,170.3992,0.9842,0.1075,0.0558,4.9938
7,71.1572,21096.0176,145.2447,0.9889,0.1479,0.7001,4.9656
8,72.5509,20077.0210,141.6934,0.9891,0.0900,0.0511,4.8828


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,68.8186,16423.6646,128.1548,0.9915,0.0895,0.0497,4.6328
1,67.0939,15697.3129,125.2889,0.9913,0.1486,0.0485,4.7797
2,74.7972,22246.6580,149.1531,0.9881,0.0964,0.0534,5.1050
3,73.9223,20654.7039,143.7174,0.9892,0.0962,0.0548,5.1506
4,69.5322,17671.2229,132.9332,0.9904,0.0819,0.0477,4.6114
5,67.6027,13357.5153,115.5747,0.9927,0.0830,0.0493,4.8440
6,73.7037,28738.1104,169.5232,0.9844,0.1069,0.0554,4.9710
7,70.9389,21054.8529,145.1029,0.9889,0.1476,0.6928,4.9521
8,72.1717,19773.3797,140.6178,0.9893,0.0896,0.0509,4.8674


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,68.5719,16162.1105,127.1303,0.9916,0.0891,0.0495,4.6215
1,66.9604,15727.1427,125.4079,0.9913,0.1486,0.0485,4.7731
2,74.5325,21999.3637,148.3218,0.9883,0.0961,0.0533,5.0935
3,73.5587,20477.9845,143.1013,0.9893,0.0958,0.0545,5.1307
4,69.3304,17684.8664,132.9845,0.9903,0.0819,0.0475,4.5998
5,67.3740,13264.6321,115.1722,0.9927,0.0829,0.0492,4.8340
6,73.3309,28476.3473,168.7494,0.9846,0.1065,0.0551,4.9503
7,70.7592,21042.2548,145.0595,0.9890,0.1472,0.6854,4.9405
8,71.8329,19493.5607,139.6193,0.9894,0.0893,0.0507,4.8544


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,68.3540,15929.1923,126.2109,0.9917,0.0887,0.0493,4.6116
1,66.8622,15774.3875,125.5961,0.9912,0.1487,0.0484,4.7687
2,74.3189,21787.1937,147.6049,0.9884,0.0958,0.0531,5.0847
3,73.2321,20331.6673,142.5892,0.9894,0.0955,0.0543,5.1131
4,69.1790,17725.4350,133.1369,0.9903,0.0819,0.0475,4.5918
5,67.1741,13194.7918,114.8686,0.9927,0.0828,0.0491,4.8256
6,73.0340,28250.6017,168.0792,0.9847,0.1060,0.0548,4.9335
7,70.6264,21058.2232,145.1145,0.9889,0.1469,0.6781,4.9313
8,71.5272,19237.5641,138.6995,0.9896,0.0890,0.0505,4.8430


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,68.1688,15724.9098,125.3990,0.9919,0.0883,0.0492,4.6037
1,66.7918,15839.0471,125.8533,0.9912,0.1487,0.0484,4.7662
2,74.1566,21610.1480,147.0039,0.9885,0.0956,0.0530,5.0787
3,72.9633,20215.7525,142.1821,0.9895,0.0951,0.0541,5.1000
4,69.0772,17792.9286,133.3901,0.9903,0.0820,0.0474,4.5864
5,67.0043,13147.9945,114.6647,0.9928,0.0828,0.0490,4.8188
6,72.7742,28060.8735,167.5138,0.9848,0.1056,0.0546,4.9193
7,70.5210,21102.7583,145.2679,0.9889,0.1467,0.6708,4.9235
8,71.2664,19005.3898,137.8600,0.9897,0.0888,0.0504,4.8337


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,68.0134,15549.2630,124.6967,0.9919,0.0880,0.0490,4.5977
1,66.7633,15921.1216,126.1789,0.9912,0.1488,0.0484,4.7658
2,74.0360,21468.2268,146.5204,0.9886,0.0954,0.0529,5.0754
3,72.7479,20130.2399,141.8811,0.9895,0.0948,0.0540,5.0906
4,69.0166,17887.3474,133.7436,0.9902,0.0821,0.0474,4.5836
5,66.8933,13124.2401,114.5611,0.9928,0.0828,0.0490,4.8149
6,72.5628,27907.1628,167.0544,0.9849,0.1052,0.0544,4.9079
7,70.4501,21175.8600,145.5193,0.9889,0.1464,0.6635,4.9175
8,71.0519,18797.0379,137.1023,0.9898,0.0886,0.0503,4.8269


In [66]:
et_model.get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 6450,
 'verbose': 0,
 'warm_start': False}

In [67]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 6450,
 'verbose': 0,
 'warm_start': False}

In [69]:
from pycaret.regression import *
from sklearn.ensemble import ExtraTreesRegressor

X_train, X_valid, y_train, y_valid = train_test_split(clust1_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1), clust1_train_df['전력소비량(kWh)'], test_size=168*clust1_train_df['건물번호'].nunique())
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 800),
    'max_depth' : trial.suggest_int('max_depth', 20, 30),
    'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_weight_fraction_leaf': 0.0,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 6450,
 'verbose': 0,
 'warm_start': False
    }

    model = ExtraTreesRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(y_valid, preds)

    return sm


study = optuna.create_study(study_name = 'ET Optuna_cluster_0')
study.optimize(objective, n_trials=30, timeout=24000)

et = study.best_trial
et_param = et.params
print('Best Trial : score {},\nparams {}'.format(et.value, et_param))

[I 2023-08-28 08:26:59,172] A new study created in memory with name: ET Optuna_cluster_0
[I 2023-08-28 08:27:00,457] Trial 0 finished with value: 5.107603398683745 and parameters: {'n_estimators': 381, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 0 with value: 5.107603398683745.
[I 2023-08-28 08:27:02,986] Trial 1 finished with value: 4.963979698753828 and parameters: {'n_estimators': 771, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 4}. Best is trial 1 with value: 4.963979698753828.
[I 2023-08-28 08:27:05,347] Trial 2 finished with value: 4.597968768639047 and parameters: {'n_estimators': 459, 'max_depth': 27, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 2 with value: 4.597968768639047.
[I 2023-08-28 08:27:06,578] Trial 3 finished with value: 5.096292422791853 and parameters: {'n_estimators': 403, 'max_depth': 23, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 2 with value: 4.597968768639047.
[I 2023-08-

Best Trial : score 4.591788552631553,
params {'n_estimators': 370, 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 1}


In [70]:
from sklearn.ensemble import RandomForestRegressor
X_train, X_valid, y_train, y_valid = train_test_split(clust1_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1), clust1_train_df['전력소비량(kWh)'], test_size=168*clust1_train_df['건물번호'].nunique())

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 10, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 5),
        'bootstrap': True,
 'ccp_alpha': 0.0,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_weight_fraction_leaf': 0.0,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 6450,
 'verbose': 1,
 'warm_start': False
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(y_valid, preds)

    
    return sm

study_0 = optuna.create_study(study_name = 'RF Optuna_0')
study_0.optimize(objective, n_trials=50, timeout=24000)

rf_0 = study_0.best_trial
rf_0_param = rf_0.params
print('Best Trial: score {},\nparams {}'.format(rf_0.value, rf_0_param))

[I 2023-08-28 08:29:44,017] A new study created in memory with name: RF Optuna_0
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 780 out of 780 | elapsed:    5.9s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 780 out of 780 | elapsed:    0.1s finished
[I 2023-08-28 08:29:50,235] Trial 0 finished with value: 5.371116116877223 and parameters: {'n_estimators': 780, 'max_depth': 28, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 0 with value: 5.371116116877223.
[Parallel(n_jobs

Best Trial: score 5.366580615454058,
params {'n_estimators': 286, 'max_depth': 25, 'min_samples_split': 4, 'min_samples_leaf': 3}


In [71]:
et_param_0 = {'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_weight_fraction_leaf': 0.0,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 6450,
 'verbose': 0,
 'warm_start': False,'n_estimators': 370, 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 1}

rf_param_0 = {'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 6450,
 'verbose': 0,
 'warm_start': False}
et_0 = ExtraTreesRegressor(**et_params_0)
rf_0 = RandomForestRegressor(**rf_params_0)

In [72]:
m_0 = [et_0, rf_0]
blender_7 = blend_models(m_0, weights=[0.9,0.1])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,69.1551,17863.9928,133.6562,0.9903,0.0916,0.0511,4.8243
1,71.8160,22224.0330,149.0773,0.9889,0.0886,0.0501,4.7656
2,69.0758,17240.0617,131.3014,0.9905,0.0898,0.0508,4.8054
3,68.1753,15876.2120,126.0008,0.9915,0.0905,0.0506,4.8574
4,65.7516,15121.0328,122.9676,0.9916,0.0777,0.0463,4.5248
5,71.4647,16878.4892,129.9172,0.9913,0.0869,0.0501,4.8094
6,67.1793,17386.9405,131.8595,0.9908,0.0914,0.0503,4.7087
7,67.8115,16150.5836,127.0849,0.9913,0.0880,0.0502,4.7790
8,66.7442,15718.5778,125.3738,0.9913,0.1498,0.0490,4.7583


In [75]:
# 만약 많이 안좋으면 et 단일모델로 간다.
# f_blender = finalize_model(blender_7)
save_model(et_0,'./ensemble-1')

# cluster2도 마찬가지로 다시해야된다.

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['CDH', 'work_time', '연면적(m2)',
                                              '냉방면적(m2)', '시간', '시간_cos', '불쾌지수',
                                              '월', '기온(C)', '시간_sin',
                                              '불쾌지수_이동평균3', '요일', '일', '습도(%)'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['건물번호', 'holiday'],
                                     transformer=SimpleImputer(strategy...
 NaN     -1
 dtype: int64}]))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['건물번호'],
                                     transformer=OneHotEncoder(cols=['건물번호'],
                                                               handle_missing='return_nan',
                                                        

# cluster2

In [79]:
selected_columns2 = ['전력소비량(kWh)','CDH',
 '건물번호',
 '냉방면적(m2)',
 '연면적(m2)',
 '시간',
 '불쾌지수',
 '월',
 '시간_sin',
 '불쾌지수_이동평균3',
 'holiday',
 '시간_cos',
 '일',
 '습도(%)',
 '요일',
 '기온(C)',
 '풍속(m/s)',
 'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns2]

clust2_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 2].reset_index(drop=True)

# con_li = ['연면적(m2)','냉방면적(m2)','시간_cos','CDH',
#           '불쾌지수','불쾌지수_이동평균3','시간',
#           '시간_sin','기온(C)','습도(%)']

# sc = StandardScaler()
# clust1_train_df[con_li] = sc.fit_transform(clust1_train_df[con_li])


In [80]:
# 앙상블


cat_li = ['건물번호',  'holiday']

# pycaret 설정
exp0 = setup(data=clust2_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)', 
             categorical_features=cat_li, train_size=len(clust2_train_df)-clust2_train_df['건물번호'].nunique()*168)
exp0.add_metric('smape','SMAPE',smape,greater_is_better=False)
# 기본 모델 생성
et_model = create_model('et')  # 선형 회귀 모델
# rf_model = create_model('rf')  # 랜덤 포레스트 모델
catboost_model = create_model('catboost')  # catboost 모델
# lightgbm_model = create_model('lightgbm')  # LightGBM 모델
# dt_model = create_model('dt')  # dt 모델
# xgboost_model = create_model('xgboost')

,Description,Value
0,Session id,6093
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(24360, 17)"
4,Transformed data shape,"(24360, 29)"
5,Transformed train set shape,"(22176, 29)"
6,Transformed test set shape,"(2184, 29)"
7,Ordinal features,1
8,Numeric features,14
9,Categorical features,2


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,30.4279,2848.1975,53.3685,0.9996,0.0346,0.0213,2.1144
1,30.9310,2995.5272,54.7314,0.9995,0.0394,0.0220,2.2034
2,30.3631,2886.1670,53.7231,0.9996,0.0383,0.0218,2.1673
3,31.2117,3398.6996,58.2984,0.9995,0.0415,0.0224,2.2345
4,30.2944,2907.3313,53.9197,0.9996,0.0331,0.0208,2.0705
5,31.4007,3152.3725,56.1460,0.9996,0.0366,0.0219,2.1846
6,33.0982,4591.1619,67.7581,0.9993,0.0451,0.0239,2.3170
7,31.8210,3381.0721,58.1470,0.9995,0.0399,0.0227,2.2487
8,35.2816,4695.3491,68.5226,0.9993,0.0404,0.0242,2.4071


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,30.7259,2048.8099,45.2638,0.9997,0.0372,0.0238,2.3670
1,31.8196,2263.2990,47.5741,0.9997,0.0414,0.0244,2.4515
2,30.7790,2236.4368,47.2910,0.9997,0.0385,0.0232,2.3199
3,30.9679,2482.7348,49.8270,0.9996,0.0419,0.0238,2.3855
4,30.7889,2061.4350,45.4030,0.9997,0.0396,0.0240,2.3927
5,31.3347,2257.1563,47.5095,0.9997,0.0394,0.0241,2.4026
6,32.5656,4253.6846,65.2203,0.9994,0.0485,0.0256,2.4867
7,32.8639,2716.9017,52.1239,0.9996,0.0421,0.0253,2.5162
8,33.6182,3199.4598,56.5638,0.9995,0.0409,0.0257,2.5586


In [30]:
et_model.get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 7291,
 'verbose': 0,
 'warm_start': False}

In [31]:
catboost_model.get_params()

{'loss_function': 'RMSE',
 'border_count': 254,
 'verbose': False,
 'task_type': 'CPU',
 'random_state': 7291}

In [140]:
m_0 = [et_model, catboost_model]
m_1 = [et_model, rf_model, catboost_model, xgboost_model]
w_li_0 = [[0.6,0.4],[0.7,0.3],[0.8,0.2],[0.9,0.1]]
w_li_1 = [[0.5,0.1,0.3,0.1],[0.4,0.2,0.3,0.1]]
b_li_0 = []
b_li_1 = []
for w in w_li_0:
    blender= blend_models(m_0, weights=w)
for w in w_li_1:
    blender= blend_models(m_1, weights=w)

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,29.1347,2588.0634,50.8730,0.9996,0.0340,0.0205,2.0376
1,27.5115,2223.1919,47.1507,0.9997,0.0377,0.0200,2.0017
2,28.7627,2369.8926,48.6815,0.9997,0.0376,0.0207,2.0798
3,29.8963,2724.7448,52.1991,0.9996,0.0339,0.0206,2.0487
4,28.2700,2483.5403,49.8351,0.9996,0.0352,0.0206,2.0557
5,28.5731,2700.6780,51.9680,0.9996,0.0339,0.0206,2.0445
6,27.8215,2126.7334,46.1165,0.9997,0.0337,0.0204,2.0385
7,29.3674,2405.3609,49.0445,0.9996,0.0332,0.0205,2.0484
8,27.9664,2493.5274,49.9352,0.9996,0.0311,0.0194,1.9366


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,29.5619,2730.0136,52.2495,0.9996,0.0345,0.0207,2.0509
1,27.8813,2331.0223,48.2807,0.9997,0.0378,0.0202,2.0171
2,29.1662,2510.0232,50.1001,0.9996,0.0379,0.0208,2.0830
3,30.1219,2854.6781,53.4292,0.9996,0.0343,0.0206,2.0532
4,28.4222,2585.1720,50.8446,0.9996,0.0352,0.0206,2.0521
5,28.9833,2851.4846,53.3993,0.9996,0.0341,0.0207,2.0568
6,28.0841,2264.3934,47.5856,0.9997,0.0340,0.0204,2.0372
7,29.8059,2574.6767,50.7413,0.9996,0.0338,0.0207,2.0679
8,28.3081,2670.4918,51.6768,0.9996,0.0315,0.0195,1.9423


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,30.2299,2912.4855,53.9674,0.9996,0.0353,0.0210,2.0838
1,28.5247,2475.2632,49.7520,0.9997,0.0383,0.0206,2.0589
2,29.7866,2689.8588,51.8638,0.9996,0.0385,0.0210,2.1067
3,30.6045,3021.0890,54.9644,0.9996,0.0349,0.0209,2.0797
4,28.8450,2723.4230,52.1864,0.9996,0.0356,0.0208,2.0701
5,29.7112,3041.4631,55.1495,0.9996,0.0346,0.0210,2.0912
6,28.6164,2444.2489,49.4393,0.9996,0.0347,0.0206,2.0604
7,30.4671,2785.4259,52.7771,0.9996,0.0347,0.0211,2.1066
8,28.9180,2887.4185,53.7347,0.9996,0.0323,0.0198,1.9687


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,31.1730,3135.4789,55.9953,0.9995,0.0364,0.0216,2.1402
1,29.4462,2655.9144,51.5356,0.9996,0.0391,0.0213,2.1231
2,30.6691,2909.3993,53.9388,0.9996,0.0394,0.0216,2.1611
3,31.3707,3223.9773,56.7801,0.9996,0.0359,0.0214,2.1287
4,29.5827,2898.2932,53.8358,0.9996,0.0362,0.0212,2.1133
5,30.7083,3270.6137,57.1893,0.9995,0.0355,0.0216,2.1492
6,29.4959,2666.3000,51.6362,0.9996,0.0357,0.0212,2.1158
7,31.3699,3037.6085,55.1145,0.9995,0.0358,0.0217,2.1643
8,29.7814,3144.3075,56.0741,0.9995,0.0335,0.0203,2.0198


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,29.6672,2682.7563,51.7953,0.9996,0.0343,0.0207,2.0597
1,27.6366,2237.9394,47.3069,0.9997,0.0380,0.0202,2.0198
2,29.0285,2415.5864,49.1486,0.9997,0.0375,0.0209,2.0924
3,30.5154,2933.6360,54.1630,0.9996,0.0346,0.0210,2.0916
4,28.6145,2575.9319,50.7536,0.9996,0.0355,0.0209,2.0833
5,29.0611,2817.5654,53.0807,0.9996,0.0343,0.0208,2.0708
6,27.9399,2136.5924,46.2233,0.9997,0.0339,0.0205,2.0501
7,29.8225,2478.5581,49.7851,0.9996,0.0336,0.0208,2.0779
8,28.2312,2541.3148,50.4115,0.9996,0.0314,0.0196,1.9510


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,30.4068,2829.0320,53.1886,0.9996,0.0348,0.0212,2.1020
1,27.9933,2278.2866,47.7314,0.9997,0.0384,0.0206,2.0526
2,29.5636,2519.8946,50.1986,0.9996,0.0379,0.0212,2.1257
3,31.2293,3099.4236,55.6725,0.9996,0.0353,0.0215,2.1404
4,29.2471,2706.6256,52.0252,0.9996,0.0361,0.0214,2.1262
5,29.7637,2922.0675,54.0562,0.9996,0.0349,0.0213,2.1185
6,28.5080,2223.1548,47.1503,0.9997,0.0345,0.0209,2.0900
7,30.6080,2626.3631,51.2481,0.9996,0.0342,0.0213,2.1213
8,28.7156,2604.9152,51.0384,0.9996,0.0320,0.0199,1.9843


In [141]:
blender_7 = blend_models(m_1,weights=[0.6,0.1,0.2,0.1])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,30.0824,2825.5905,53.1563,0.9996,0.0348,0.0209,2.0740
1,28.0885,2348.6623,48.4630,0.9997,0.0382,0.0204,2.0406
2,29.4838,2558.4808,50.5814,0.9996,0.0379,0.0209,2.0997
3,30.7817,3069.8265,55.4060,0.9996,0.0350,0.0211,2.1017
4,28.8450,2675.0726,51.7211,0.9996,0.0356,0.0209,2.0839
5,29.5737,2974.9698,54.5433,0.9996,0.0346,0.0210,2.0886
6,28.2475,2276.7706,47.7155,0.9997,0.0342,0.0206,2.0548
7,30.2869,2650.5340,51.4833,0.9996,0.0342,0.0211,2.1033
8,28.6741,2722.0224,52.1730,0.9996,0.0319,0.0197,1.9630


In [142]:
blender_8 = blend_models(m_0,weights=[0.5,0.5])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,29.0035,2486.6348,49.8662,0.9996,0.0339,0.0206,2.0470
1,27.4384,2151.7718,46.3872,0.9997,0.0378,0.0202,2.0162
2,28.6062,2269.4669,47.6389,0.9997,0.0376,0.0209,2.0978
3,29.8889,2631.2890,51.2961,0.9996,0.0339,0.0207,2.0649
4,28.3716,2418.5277,49.1785,0.9997,0.0355,0.0209,2.0817
5,28.3858,2589.0434,50.8826,0.9996,0.0340,0.0207,2.0544
6,27.8014,2031.2689,45.0696,0.9997,0.0337,0.0206,2.0596
7,29.1530,2277.4786,47.7229,0.9997,0.0330,0.0205,2.0485
8,27.8931,2356.5255,48.5441,0.9997,0.0310,0.0196,1.9508


In [143]:
blender_9 = blend_models(m_0,weights=[0.55,0.45])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,29.0394,2532.2839,50.3218,0.9996,0.0339,0.0205,2.0397
1,27.4320,2182.9306,46.7218,0.9997,0.0377,0.0201,2.0056
2,28.6610,2314.7166,48.1115,0.9997,0.0375,0.0208,2.0859
3,29.8640,2673.4572,51.7055,0.9996,0.0339,0.0206,2.0541
4,28.2863,2446.4566,49.4617,0.9997,0.0353,0.0207,2.0660
5,28.4506,2639.9642,51.3806,0.9996,0.0339,0.0206,2.0465
6,27.7812,2073.7267,45.5382,0.9997,0.0337,0.0205,2.0464
7,29.2274,2336.2406,48.3347,0.9996,0.0331,0.0205,2.0460
8,27.8949,2420.0311,49.1938,0.9996,0.0310,0.0195,1.9410


In [81]:
from catboost import CatBoostRegressor
et_param_2 = {'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 28,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 7291,
 'verbose': 0,
 'warm_start': False}
cat_param_2 = {'loss_function': 'RMSE',
 'border_count': 254,'iterations': 494,'learning_rate': 0.04511244169304455,
 'verbose': False,'depth': 13, 'l2_leaf_reg': 2.5, 'min_child_samples': 1,
 'task_type': 'CPU',
 'random_state': 7291}
et_2 = ExtraTreesRegressor(**et_param_2)
cat_2 = CatBoostRegressor(**cat_param_2)

In [62]:
# m_0 = [et_2, cat_2]
# blender_0 = blend_models(m_0,weights=[0.5,0.5])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,53.6786,7867.0962,88.6967,0.9951,0.0465,0.0281,2.7777
1,54.2125,7979.9831,89.3308,0.9950,0.0456,0.0276,2.7416
2,55.2407,8405.0031,91.6788,0.9947,0.0484,0.0289,2.8661
3,53.2348,7523.2998,86.7370,0.9953,0.0491,0.0287,2.8316
4,55.1306,8270.4140,90.9418,0.9947,0.0505,0.0293,2.9010
5,54.3955,7826.0334,88.4649,0.9950,0.0481,0.0286,2.8304
6,53.3179,7822.9549,88.4475,0.9951,0.0474,0.0283,2.7989
7,54.7437,7799.2813,88.3135,0.9950,0.0497,0.0292,2.8645
8,54.2472,7967.5373,89.2611,0.9950,0.0490,0.0288,2.8357


In [82]:
# 2.0542
# f_blender = finalize_model(blender_0)
save_model(et_2,'./ensemble-2-et')
save_model(cat_2,'./ensemble-2-cat')
# or
# # save_model(b_li_1[num],'./ensemble-2')
# or
# # save_model(blender_7,'./ensemble-2')
# or
# # save_model(blender_8,'./ensemble-2')
# or
# # save_model(blender_8,'./ensemble-2')

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['CDH', '냉방면적(m2)', '연면적(m2)', '시간',
                                              '불쾌지수', '월', '시간_sin',
                                              '불쾌지수_이동평균3', '시간_cos', '일',
                                              '습도(%)', '요일', '기온(C)',
                                              '풍속(m/s)'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['건물번호', 'holiday'],
                                     transformer=SimpleImputer(strategy='...
                                                                          'data_type': dtype('O'),
                                                                          'mapping': False    0
 True     1
 NaN     -1
 dtype: int64}]))),
                 ('onehot_encoding',
                  Transforme

# clust3

In [83]:
selected_columns3 = ['전력소비량(kWh)',
'work_time',
 '연면적(m2)',
 '냉방면적(m2)',
 '시간',
 '건물번호',
 '시간_cos',
 '불쾌지수',
 '월',
 '기온(C)',
 'holiday',
 '시간_sin',
 '불쾌지수_이동평균3',
 '요일',
 '일',
 '습도(%)',
 '태양광용량(kW)','CDH',
'km_cluster']
combined_clust_train_df = combined_clust_df[selected_columns3]

clust3_train_df = combined_clust_train_df[combined_clust_train_df['km_cluster'] == 3].reset_index(drop=True)

# con_li = ['연면적(m2)','냉방면적(m2)','시간_cos','CDH',
#           '불쾌지수','불쾌지수_이동평균3','시간','태양광용량(kW)','CDH',
#           '시간_sin','기온(C)','습도(%)']

# sc = StandardScaler()
# clust3_train_df[con_li] = sc.fit_transform(clust3_train_df[con_li])

In [84]:

cat_li = ['건물번호',  'holiday','work_time']

# pycaret 설정
exp0 = setup(data=clust3_train_df.drop(columns=['km_cluster']), target='전력소비량(kWh)', 
             categorical_features=cat_li, train_size=len(clust3_train_df)-clust3_train_df['건물번호'].nunique()*168)
exp0.add_metric('smape','SMAPE',smape,greater_is_better=False)
et_model = create_model('et')
rf_model = create_model('rf')  # 랜덤 포레스트 모델
# catboost_model = create_model('catboost')  # catboost 모델
# lightgbm_model = create_model('lightgbm')  # LightGBM 모델
# dt_model = create_model('dt')  # dt 모델
# xgboost_model = create_model('xgboost')

,Description,Value
0,Session id,7073
1,Target,전력소비량(kWh)
2,Target type,Regression
3,Original data shape,"(65280, 18)"
4,Transformed data shape,"(65280, 18)"
5,Transformed train set shape,"(59904, 18)"
6,Transformed test set shape,"(5376, 18)"
7,Ordinal features,2
8,Numeric features,14
9,Categorical features,3


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,54.5987,9089.6630,95.3397,0.9943,0.0492,0.0280,2.7636
1,54.9581,9143.4609,95.6214,0.9944,0.0509,0.0284,2.8236
2,54.8177,9274.0068,96.3016,0.9943,0.0491,0.0280,2.7585
3,54.0417,8640.5084,92.9543,0.9946,0.0511,0.0286,2.8224
4,54.4697,8655.8341,93.0367,0.9944,0.0525,0.0289,2.8389
5,54.7298,9345.6316,96.6728,0.9942,0.0494,0.0278,2.7669
6,56.1653,9729.7603,98.6395,0.9936,0.0516,0.0287,2.8637
7,55.0376,9497.5278,97.4553,0.9941,0.0521,0.0283,2.7999
8,54.6910,8450.6290,91.9273,0.9945,0.0504,0.0286,2.8209


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,61.1010,10504.3441,102.4907,0.9934,0.0540,0.0313,3.0893
1,64.6711,12328.5709,111.0341,0.9924,0.0577,0.0336,3.3174
2,63.0366,11567.0905,107.5504,0.9929,0.0555,0.0322,3.1593
3,60.8026,10559.3292,102.7586,0.9933,0.0559,0.0320,3.1647
4,62.9032,11150.0175,105.5936,0.9928,0.0601,0.0335,3.2779
5,62.7846,12071.2917,109.8694,0.9925,0.0567,0.0319,3.1785
6,63.8419,11964.4101,109.3819,0.9922,0.0570,0.0328,3.2565
7,62.1813,11102.6537,105.3691,0.9931,0.0564,0.0318,3.1521
8,61.3427,10065.3538,100.3262,0.9935,0.0542,0.0318,3.1384


In [157]:
m_0 = [et_model, rf_model]
m_1 = [et_model, rf_model, dt_model]
w_li_0 = [[0.6,0.4],[0.7,0.3],[0.8,0.2],[0.9,0.1]]
w_li_1 = [[0.5,0.3,0.2],[0.5,0.4,0.1],[0.6,0.3,0.1],[0.7,0.2,0.1],[0.8,0.1,0.1]]
b_li_0 = []
b_li_1 = []
for w in w_li_0 :
    blender = blend_models(m_0,weights=w)
    b_li_0.append(blender)
for w in w_li_1:
    blender = blend_models(m_1,weights=w)
    b_li_1.append(blender)

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,55.5394,8710.1076,93.3280,0.9945,0.0494,0.0286,2.8410
1,56.2107,9219.1520,96.0164,0.9943,0.0527,0.0294,2.8860
2,57.9048,10460.0403,102.2743,0.9935,0.0527,0.0300,2.9530
3,55.9698,9512.8330,97.5338,0.9940,0.0532,0.0290,2.8699
4,56.3501,8651.9473,93.0158,0.9944,0.0496,0.0289,2.8664
5,55.4591,8929.2251,94.4946,0.9943,0.0508,0.0289,2.8455
6,54.0320,8261.3225,90.8918,0.9950,0.0488,0.0282,2.7930
7,56.9358,9388.3106,96.8933,0.9938,0.0509,0.0297,2.9361
8,56.2163,9262.0345,96.2395,0.9941,0.0516,0.0292,2.8976


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,54.9006,8552.4150,92.4793,0.9946,0.0489,0.0283,2.8038
1,55.6270,9097.6839,95.3818,0.9943,0.0523,0.0290,2.8538
2,57.4152,10390.0295,101.9315,0.9936,0.0525,0.0297,2.9283
3,55.4072,9402.3802,96.9659,0.9940,0.0529,0.0287,2.8410
4,55.7145,8502.8808,92.2111,0.9945,0.0492,0.0285,2.8325
5,54.8477,8762.6336,93.6089,0.9945,0.0502,0.0285,2.8108
6,53.3997,8146.1778,90.2562,0.9950,0.0483,0.0278,2.7592
7,56.2745,9266.5422,96.2629,0.9938,0.0505,0.0293,2.9031
8,55.5739,9083.0751,95.3052,0.9942,0.0512,0.0289,2.8673


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,54.3852,8447.2861,91.9091,0.9947,0.0484,0.0279,2.7726
1,55.1647,9025.4457,95.0023,0.9944,0.0520,0.0288,2.8272
2,57.0723,10371.3323,101.8397,0.9936,0.0526,0.0296,2.9114
3,55.0008,9346.4565,96.6771,0.9941,0.0527,0.0284,2.8197
4,55.2116,8408.3792,91.6972,0.9946,0.0489,0.0283,2.8053
5,54.3954,8658.9923,93.0537,0.9945,0.0498,0.0282,2.7847
6,52.8931,8083.3627,89.9075,0.9951,0.0480,0.0275,2.7319
7,55.7710,9198.0628,95.9065,0.9939,0.0503,0.0291,2.8774
8,55.0524,8954.7508,94.6295,0.9943,0.0508,0.0287,2.8430


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,54.0162,8394.7207,91.6227,0.9947,0.0482,0.0277,2.7491
1,54.8741,9002.4377,94.8812,0.9944,0.0519,0.0286,2.8099
2,56.9201,10403.9486,101.9997,0.9936,0.0527,0.0295,2.9045
3,54.7441,9345.0620,96.6699,0.9941,0.0527,0.0283,2.8059
4,54.8745,8368.4425,91.4792,0.9946,0.0487,0.0281,2.7865
5,54.1203,8618.3014,92.8348,0.9945,0.0495,0.0281,2.7677
6,52.5557,8072.8771,89.8492,0.9951,0.0478,0.0274,2.7128
7,55.4574,9182.8722,95.8273,0.9939,0.0503,0.0289,2.8604
8,54.6775,8877.0614,94.2182,0.9944,0.0507,0.0285,2.8265


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,56.4635,9147.3678,95.6419,0.9942,0.0507,0.0291,2.8896
1,57.5906,9671.4587,98.3436,0.9940,0.0533,0.0300,2.9488
2,59.0665,10952.2692,104.6531,0.9932,0.0534,0.0304,3.0048
3,57.0049,9960.3676,99.8016,0.9937,0.0548,0.0295,2.9287
4,56.8319,8928.8466,94.4926,0.9943,0.0513,0.0293,2.9033
5,56.7452,9495.7502,97.4461,0.9940,0.0525,0.0294,2.9036
6,55.1586,8833.1386,93.9848,0.9946,0.0502,0.0287,2.8412
7,58.3489,10007.9783,100.0399,0.9933,0.0524,0.0303,2.9963
8,57.7840,9928.5305,99.6420,0.9937,0.0535,0.0299,2.9643


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,56.0789,8928.3698,94.4901,0.9944,0.0501,0.0290,2.8738
1,56.9035,9411.6693,97.0138,0.9941,0.0529,0.0297,2.9214
2,58.5178,10658.4105,103.2396,0.9934,0.0529,0.0302,2.9808
3,56.5091,9709.1417,98.5350,0.9938,0.0540,0.0293,2.9016
4,56.6892,8773.0476,93.6645,0.9944,0.0504,0.0291,2.8923
5,56.2008,9219.3193,96.0173,0.9942,0.0517,0.0292,2.8819
6,54.7422,8523.7152,92.3240,0.9948,0.0495,0.0285,2.8269
7,57.7222,9670.4221,98.3383,0.9936,0.0516,0.0300,2.9710
8,57.0925,9583.7347,97.8966,0.9939,0.0525,0.0296,2.9352


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,55.3358,8725.6964,93.4114,0.9945,0.0495,0.0285,2.8322
1,56.2282,9241.0699,96.1305,0.9942,0.0524,0.0293,2.8844
2,57.9428,10542.5354,102.6768,0.9935,0.0527,0.0299,2.9512
3,55.8493,9552.4753,97.7368,0.9939,0.0536,0.0289,2.8674
4,55.9768,8577.0236,92.6122,0.9945,0.0499,0.0288,2.8547
5,55.4678,8994.6374,94.8401,0.9943,0.0510,0.0288,2.8412
6,53.9951,8362.2691,91.4454,0.9949,0.0490,0.0281,2.7873
7,56.9600,9503.4390,97.4856,0.9937,0.0511,0.0296,2.9334
8,56.3505,9363.8456,96.7670,0.9941,0.0520,0.0293,2.9003


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,54.7253,8575.5867,92.6045,0.9946,0.0489,0.0282,2.7969
1,55.6821,9119.7005,95.4971,0.9943,0.0520,0.0290,2.8535
2,57.5086,10477.9739,102.3620,0.9935,0.0526,0.0297,2.9293
3,55.3337,9450.3380,97.2128,0.9940,0.0533,0.0286,2.8402
4,55.4058,8435.5644,91.8453,0.9946,0.0494,0.0284,2.8240
5,54.8774,8832.9058,93.9835,0.9944,0.0504,0.0285,2.8079
6,53.3880,8253.1526,90.8469,0.9950,0.0486,0.0278,2.7547
7,56.3690,9389.7448,96.9007,0.9938,0.0508,0.0293,2.9040
8,55.7540,9194.5915,95.8884,0.9942,0.0516,0.0290,2.8720


,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,54.2564,8478.0406,92.0763,0.9946,0.0485,0.0279,2.7686
1,55.2806,9047.5612,95.1187,0.9944,0.0517,0.0288,2.8301
2,57.2223,10464.7260,102.2972,0.9935,0.0526,0.0296,2.9149
3,54.9715,9402.7299,96.9677,0.9940,0.0531,0.0284,2.8212
4,54.9794,8348.6701,91.3711,0.9946,0.0491,0.0282,2.8011
5,54.4629,8734.1246,93.4565,0.9945,0.0500,0.0282,2.7844
6,52.9296,8196.3656,90.5338,0.9950,0.0483,0.0275,2.7297
7,55.9620,9329.3395,96.5885,0.9938,0.0506,0.0291,2.8831
8,55.2964,9075.9724,95.2679,0.9942,0.0513,0.0287,2.8511


In [158]:
blend_10 = blend_models(m_0,weights=[0.93,0.07])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,53.9344,8389.2011,91.5926,0.9947,0.0481,0.0277,2.7440
1,54.8217,9005.1351,94.8954,0.9944,0.0519,0.0286,2.8068
2,56.9098,10423.7397,102.0967,0.9935,0.0528,0.0295,2.9044
3,54.6993,9355.2769,96.7227,0.9941,0.0527,0.0283,2.8032
4,54.8111,8367.1017,91.4719,0.9946,0.0487,0.0280,2.7830
5,54.0808,8618.3694,92.8352,0.9945,0.0495,0.0280,2.7647
6,52.4934,8079.9357,89.8885,0.9951,0.0478,0.0273,2.7089
7,55.3983,9188.7063,95.8577,0.9939,0.0503,0.0289,2.8570
8,54.5967,8863.6285,94.1468,0.9944,0.0506,0.0285,2.8232


In [87]:
et_model.get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 7073,
 'verbose': 0,
 'warm_start': False}

In [88]:
rf_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 7073,
 'verbose': 0,
 'warm_start': False}

In [ ]:
from pycaret.regression import *
from sklearn.ensemble import ExtraTreesRegressor

X_train, X_valid, y_train, y_valid = train_test_split(clust3_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1), clust3_train_df['전력소비량(kWh)'], test_size=168*clust3_train_df['건물번호'].nunique())
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 800),
    'max_depth' : trial.suggest_int('max_depth', 20, 30),
    'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_weight_fraction_leaf': 0.0,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 7073,
 'verbose': 0,
 'warm_start': False
    }

    model = ExtraTreesRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(y_valid, preds)

    return sm


study = optuna.create_study(study_name = 'ET Optuna_cluster_0')
study.optimize(objective, n_trials=30, timeout=24000)

et = study.best_trial
et_param = et.params
print('Best Trial : score {},\nparams {}'.format(et.value, et_param))


[I 2023-08-28 08:48:56,285] A new study created in memory with name: ET Optuna_cluster_0
[I 2023-08-28 08:49:02,696] Trial 0 finished with value: 2.873668324680523 and parameters: {'n_estimators': 786, 'max_depth': 23, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: 2.873668324680523.
[I 2023-08-28 08:49:07,747] Trial 1 finished with value: 3.2050981701349692 and parameters: {'n_estimators': 793, 'max_depth': 28, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.873668324680523.
[I 2023-08-28 08:49:12,942] Trial 2 finished with value: 3.0572029067429045 and parameters: {'n_estimators': 767, 'max_depth': 23, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.873668324680523.
[I 2023-08-28 08:49:16,161] Trial 3 finished with value: 3.2034716053266243 and parameters: {'n_estimators': 508, 'max_depth': 30, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.873668324680523.
[I 2023-

Best Trial : score 2.842359426404162,
params {'n_estimators': 384, 'max_depth': 29, 'min_samples_split': 3, 'min_samples_leaf': 1}


In [90]:

from sklearn.ensemble import RandomForestRegressor
X_train, X_valid, y_train, y_valid = train_test_split(clust3_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1), clust3_train_df['전력소비량(kWh)'], test_size=168*clust3_train_df['건물번호'].nunique())

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'criterion': 'squared_error',
        'max_depth' : trial.suggest_int('max_depth', 10, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 3, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 3, 5),
'bootstrap': True,
 'ccp_alpha': 0.0,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_weight_fraction_leaf': 0.0,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 7073,
 'verbose': 0,
 'warm_start': False
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train,y_train)

    preds = model.predict(X_valid)

    sm = smape(y_valid, preds)

    
    return sm

study_0 = optuna.create_study(study_name = 'RF Optuna_0')
study_0.optimize(objective, n_trials=50, timeout=24000)

rf_0 = study_0.best_trial
rf_0_param = rf_0.params
print('Best Trial: score {},\nparams {}'.format(rf_0.value, rf_0_param))



[I 2023-08-28 08:51:00,972] A new study created in memory with name: RF Optuna_0
[I 2023-08-28 08:51:11,681] Trial 0 finished with value: 4.0211250109401835 and parameters: {'n_estimators': 768, 'max_depth': 14, 'min_samples_split': 9, 'min_samples_leaf': 3}. Best is trial 0 with value: 4.0211250109401835.
[I 2023-08-28 08:51:21,860] Trial 1 finished with value: 3.311297069322465 and parameters: {'n_estimators': 616, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 1 with value: 3.311297069322465.
[I 2023-08-28 08:51:33,823] Trial 2 finished with value: 3.2879580027869415 and parameters: {'n_estimators': 717, 'max_depth': 29, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 2 with value: 3.2879580027869415.
[I 2023-08-28 08:51:48,647] Trial 3 finished with value: 3.3830956894715394 and parameters: {'n_estimators': 934, 'max_depth': 19, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 2 with value: 3.2879580027869415.
[I 2023-08-28

KeyboardInterrupt: 

In [ ]:
et_model


In [182]:
m_0 = [et_model, rf_model]
blend_10 = blend_models(m_0,weights=[0.95,0.05])

,MAE,MSE,RMSE,R2,RMSLE,MAPE,SMAPE
Fold,,,,,,,
0,55.2128,8963.2016,94.6742,0.9944,0.0495,0.0286,2.8281
1,55.5818,9310.9618,96.4933,0.9943,0.0505,0.0285,2.8157
2,54.8400,8610.6113,92.7934,0.9946,0.0500,0.0288,2.8510
3,54.3638,8970.1580,94.7109,0.9942,0.0519,0.0286,2.8151
4,55.2090,8780.6619,93.7052,0.9944,0.0477,0.0282,2.8029
5,53.5003,9225.8561,96.0513,0.9943,0.0515,0.0281,2.7712
6,54.2346,8230.2046,90.7205,0.9948,0.0495,0.0281,2.7947
7,54.3974,8670.3331,93.1146,0.9945,0.0508,0.0285,2.8065
8,54.8159,8919.6782,94.4440,0.9944,0.0509,0.0284,2.7955


In [183]:
# 2.797
# save_model(b_li_0[num],'./ensemble-3') # 
# or
# save_model(b_li_1[num],'./ensemble-3')
# or
f_blend = finalize_model(blend_10)
save_model(f_blend,'./ensemble-3')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['연면적(m2)', '냉방면적(m2)', '시간',
                                              '시간_cos', '불쾌지수', '월', '기온(C)',
                                              '시간_sin', '불쾌지수_이동평균3', '요일', '일',
                                              '습도(%)', '태양광용량(kW)', 'CDH'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['건물번호', 'holiday', 'work_time'],
                                     transformer=SimpleImpu...
                                     transformer=TargetEncoder(cols=[],
                                                               handle_missing='return_nan'))),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('actual_estimator',
                  VotingRegressor

# 예측

In [ ]:
selected_columns0.append('건물유형_라벨')
selected_columns1.append('건물유형_라벨')
selected_columns2.append('건물유형_라벨')
selected_columns3.append('건물유형_라벨')

rm_list = ['전력소비량(kWh)', 'km_cluster']
for value in rm_list:
    selected_columns0.remove(value)
    selected_columns1.remove(value)
    selected_columns2.remove(value)
    selected_columns3.remove(value)


test_df_list = [test_df[selected_columns0],test_df[selected_columns1],test_df[selected_columns2],test_df[selected_columns3]]

In [100]:
selected_columns0.append('건물유형_라벨')
selected_columns1.append('건물유형_라벨')
selected_columns2.append('건물유형_라벨')
selected_columns3.append('건물유형_라벨')

rm_list = ['전력소비량(kWh)', 'km_cluster']
for value in rm_list:
    selected_columns0.remove(value)
    selected_columns1.remove(value)
    selected_columns2.remove(value)
    selected_columns3.remove(value)


test_df_list = [test_df[selected_columns0],test_df[selected_columns1],test_df[selected_columns2],test_df[selected_columns3]]

In [103]:
clust_to_num = {0: [1, 3, 9, 11], 1: [5, 12], 2: [4, 8], 3: [2, 6, 7, 10]}

test_df['answer'] = np.nan

et_params_0= {'n_estimators':1000, 'max_depth': 29, 'min_samples_split': 4, 'min_samples_leaf': 1,
       'max_features': 1.0,'criterion': 'squared_error', 'random_state': 5568}
et_0 = ExtraTreesRegressor(**et_params_0)
X = clust0_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1)
y = clust0_train_df['전력소비량(kWh)']
et_0.fit(X,y)



# 건물유형라벨이 해당 클러스터에 속하는 테스트 데이터 선택
cluster_test_data = test_df_list[0][test_df_list[0]['건물유형_라벨'].isin([1,3,9,11])]
cluster_test_data = cluster_test_data.drop(columns=['건물유형_라벨'])
index_list = cluster_test_data.index.tolist()

preds = et_0.predict(cluster_test_data)

# index_list와 predictions를 이용하여 데이터프레임 생성
answer_df = pd.DataFrame({'answer': preds}, index=index_list)

# 예측 결과를 원래 데이터프레임에 추가
test_df.loc[answer_df.index, 'answer'] = answer_df['answer']


test_df

,건물번호,일시,기온(C),풍속(m/s),습도(%),불쾌지수,불쾌지수_이동평균3,불쾌지수_이동평균5,holiday,월,...,지식산업센터,할인마트,호텔및리조트,건물유형_라벨,태양광여부,work_time,low_day,particular,CDH,answer
0,1,2022-08-25 00:00:00,23.5,2.2,72,71.78980,71.789800,71.789800,False,8,...,0,0,0,1,False,0,0,0,-2.5,2072.395590
1,1,2022-08-25 01:00:00,23.0,0.9,72,71.02840,71.409100,71.409100,False,8,...,0,0,0,1,False,0,0,0,-5.5,2106.334758
2,1,2022-08-25 02:00:00,22.7,1.5,75,70.81675,71.211650,71.211650,False,8,...,0,0,0,1,False,0,0,0,-8.8,1926.321013
3,1,2022-08-25 03:00:00,22.1,1.3,78,70.11262,70.652590,70.936892,False,8,...,0,0,0,1,False,0,0,0,-12.7,1915.876200
4,1,2022-08-25 04:00:00,21.8,1.0,77,69.56514,70.164837,70.662542,False,8,...,0,0,0,1,False,0,0,0,-16.9,1938.749400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,2022-08-31 19:00:00,22.5,0.9,84,71.22400,72.378973,72.792244,False,8,...,0,0,1,12,False,1,0,0,-34.5,NaN
16796,100,2022-08-31 20:00:00,20.7,0.4,95,68.95035,70.961363,71.899884,False,8,...,0,0,1,12,False,1,0,0,-34.4,NaN
16797,100,2022-08-31 21:00:00,20.2,0.4,98,68.24604,69.473463,70.866662,False,8,...,0,0,1,12,False,1,0,0,-35.3,NaN
16798,100,2022-08-31 22:00:00,20.1,1.1,97,68.01203,68.402807,69.828432,False,8,...,0,0,1,12,False,1,0,0,-36.8,NaN


In [104]:
et_param = {'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_weight_fraction_leaf': 0.0,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 6450,
 'verbose': 0,
 'warm_start': False,'n_estimators': 370, 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 1}
et_1 = ExtraTreesRegressor(**et_param)
X = clust1_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1)
y = clust1_train_df['전력소비량(kWh)']
et_1.fit(X,y)



# 건물유형라벨이 해당 클러스터에 속하는 테스트 데이터 선택
cluster_test_data = test_df_list[1][test_df_list[1]['건물유형_라벨'].isin([5,12])]
cluster_test_data = cluster_test_data.drop(columns=['건물유형_라벨'])
index_list = cluster_test_data.index.tolist()

preds = et_1.predict(cluster_test_data)

# index_list와 predictions를 이용하여 데이터프레임 생성
answer_df = pd.DataFrame({'answer': preds}, index=index_list)

# 예측 결과를 원래 데이터프레임에 추가
test_df.loc[answer_df.index, 'answer'] = answer_df['answer']

In [105]:
test_df

,건물번호,일시,기온(C),풍속(m/s),습도(%),불쾌지수,불쾌지수_이동평균3,불쾌지수_이동평균5,holiday,월,...,지식산업센터,할인마트,호텔및리조트,건물유형_라벨,태양광여부,work_time,low_day,particular,CDH,answer
0,1,2022-08-25 00:00:00,23.5,2.2,72,71.78980,71.789800,71.789800,False,8,...,0,0,0,1,False,0,0,0,-2.5,2072.395590
1,1,2022-08-25 01:00:00,23.0,0.9,72,71.02840,71.409100,71.409100,False,8,...,0,0,0,1,False,0,0,0,-5.5,2106.334758
2,1,2022-08-25 02:00:00,22.7,1.5,75,70.81675,71.211650,71.211650,False,8,...,0,0,0,1,False,0,0,0,-8.8,1926.321013
3,1,2022-08-25 03:00:00,22.1,1.3,78,70.11262,70.652590,70.936892,False,8,...,0,0,0,1,False,0,0,0,-12.7,1915.876200
4,1,2022-08-25 04:00:00,21.8,1.0,77,69.56514,70.164837,70.662542,False,8,...,0,0,0,1,False,0,0,0,-16.9,1938.749400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,2022-08-31 19:00:00,22.5,0.9,84,71.22400,72.378973,72.792244,False,8,...,0,0,1,12,False,1,0,0,-34.5,869.229568
16796,100,2022-08-31 20:00:00,20.7,0.4,95,68.95035,70.961363,71.899884,False,8,...,0,0,1,12,False,1,0,0,-34.4,758.454486
16797,100,2022-08-31 21:00:00,20.2,0.4,98,68.24604,69.473463,70.866662,False,8,...,0,0,1,12,False,1,0,0,-35.3,706.614486
16798,100,2022-08-31 22:00:00,20.1,1.1,97,68.01203,68.402807,69.828432,False,8,...,0,0,1,12,False,1,0,0,-36.8,576.212270


In [107]:
et_param_2 = {'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 28,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 7291,
 'verbose': 0,
 'warm_start': False}
cat_param_2 = {'loss_function': 'RMSE',
 'border_count': 254,'iterations': 494,'learning_rate': 0.04511244169304455,
 'verbose': False,'depth': 13, 'l2_leaf_reg': 2.5, 'min_child_samples': 1,
 'task_type': 'CPU',
 'random_state': 7291}
et_2 = ExtraTreesRegressor(**et_param_2)
cat_2 = CatBoostRegressor(**cat_param_2)

X = clust2_train_df.drop(['km_cluster','전력소비량(kWh)'],axis=1)
y = clust2_train_df['전력소비량(kWh)']
et_2.fit(X,y)
cat_2.fit(X,y)



# 건물유형라벨이 해당 클러스터에 속하는 테스트 데이터 선택
cluster_test_data = test_df_list[2][test_df_list[2]['건물유형_라벨'].isin([4,8])]
cluster_test_data = cluster_test_data.drop(columns=['건물유형_라벨'])
index_list = cluster_test_data.index.tolist()

preds_et = et_2.predict(cluster_test_data)
preds_cat = cat_2.predict(cluster_test_data)
preds = (preds_et + preds_cat)/2

# index_list와 predictions를 이용하여 데이터프레임 생성
answer_df = pd.DataFrame({'answer': preds}, index=index_list)

# 예측 결과를 원래 데이터프레임에 추가
test_df.loc[answer_df.index, 'answer'] = answer_df['answer']

In [ ]:
clust_to_num = {0: [1, 3, 9, 11], 1: [5, 12], 2: [4, 8], 3: [2, 6, 7, 10]}

In [108]:
cluster_model = load_model('ensemble-3')

# 건물유형라벨이 해당 클러스터에 속하는 테스트 데이터 선택
cluster_test_data = test_df_list[3][test_df_list[3]['건물유형_라벨'].isin([2,6,7,10])]
cluster_test_data = cluster_test_data.drop(columns=['건물유형_라벨'])
index_list = cluster_test_data.index.tolist()

# 불러온 모델로 예측 수행
predictions = predict_model(cluster_model, cluster_test_data)

# index_list와 predictions를 이용하여 데이터프레임 생성
answer_df = pd.DataFrame({'answer': predictions['prediction_label']}, index=index_list)

# 예측 결과를 원래 데이터프레임에 추가
test_df.loc[answer_df.index, 'answer'] = answer_df['answer']


test_df

Transformation Pipeline and Model Successfully Loaded


,건물번호,일시,기온(C),풍속(m/s),습도(%),불쾌지수,불쾌지수_이동평균3,불쾌지수_이동평균5,holiday,월,...,지식산업센터,할인마트,호텔및리조트,건물유형_라벨,태양광여부,work_time,low_day,particular,CDH,answer
0,1,2022-08-25 00:00:00,23.5,2.2,72,71.78980,71.789800,71.789800,False,8,...,0,0,0,1,False,0,0,0,-2.5,2072.395590
1,1,2022-08-25 01:00:00,23.0,0.9,72,71.02840,71.409100,71.409100,False,8,...,0,0,0,1,False,0,0,0,-5.5,2106.334758
2,1,2022-08-25 02:00:00,22.7,1.5,75,70.81675,71.211650,71.211650,False,8,...,0,0,0,1,False,0,0,0,-8.8,1926.321013
3,1,2022-08-25 03:00:00,22.1,1.3,78,70.11262,70.652590,70.936892,False,8,...,0,0,0,1,False,0,0,0,-12.7,1915.876200
4,1,2022-08-25 04:00:00,21.8,1.0,77,69.56514,70.164837,70.662542,False,8,...,0,0,0,1,False,0,0,0,-16.9,1938.749400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,2022-08-31 19:00:00,22.5,0.9,84,71.22400,72.378973,72.792244,False,8,...,0,0,1,12,False,1,0,0,-34.5,869.229568
16796,100,2022-08-31 20:00:00,20.7,0.4,95,68.95035,70.961363,71.899884,False,8,...,0,0,1,12,False,1,0,0,-34.4,758.454486
16797,100,2022-08-31 21:00:00,20.2,0.4,98,68.24604,69.473463,70.866662,False,8,...,0,0,1,12,False,1,0,0,-35.3,706.614486
16798,100,2022-08-31 22:00:00,20.1,1.1,97,68.01203,68.402807,69.828432,False,8,...,0,0,1,12,False,1,0,0,-36.8,576.212270


In [186]:

# 예측값을 저장할 컬럼
test_df['answer'] = np.nan


# 각 클러스터 모델에 대해 예측 수행
for cluster, building_labels in clust_to_num


.items():
    # 해당 클러스터의 모델 불러오기
    cluster_model = load_model(path + 'ensemble-' + str(cluster))

    # 건물유형라벨이 해당 클러스터에 속하는 테스트 데이터 선택
    cluster_test_data = test_df_list[cluster][test_df_list[cluster]['건물유형_라벨'].isin(building_labels)]
    cluster_test_data = cluster_test_data.drop(columns=['건물유형_라벨'])
    index_list = cluster_test_data.index.tolist()

    # 불러온 모델로 예측 수행
    predictions = predict_model(cluster_model, cluster_test_data)

    # index_list와 predictions를 이용하여 데이터프레임 생성
    answer_df = pd.DataFrame({'answer': predictions['prediction_label']}, index=index_list)

    # 예측 결과를 원래 데이터프레임에 추가
    test_df.loc[answer_df.index, 'answer'] = answer_df['answer']


test_df

Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


Transformation Pipeline and Model Successfully Loaded


,건물번호,일시,기온(C),풍속(m/s),습도(%),불쾌지수,불쾌지수_이동평균3,불쾌지수_이동평균5,holiday,월,...,지식산업센터,할인마트,호텔및리조트,건물유형_라벨,태양광여부,work_time,low_day,particular,CDH,answer
0,1,2022-08-25 00:00:00,23.5,2.2,72,71.78980,71.789800,71.789800,False,8,...,0,0,0,1,False,0,0,0,-2.5,6856.975674
1,1,2022-08-25 01:00:00,23.0,0.9,72,71.02840,71.409100,71.409100,False,8,...,0,0,0,1,False,0,0,0,-5.5,7018.911182
2,1,2022-08-25 02:00:00,22.7,1.5,75,70.81675,71.211650,71.211650,False,8,...,0,0,0,1,False,0,0,0,-8.8,6880.012286
3,1,2022-08-25 03:00:00,22.1,1.3,78,70.11262,70.652590,70.936892,False,8,...,0,0,0,1,False,0,0,0,-12.7,6875.604845
4,1,2022-08-25 04:00:00,21.8,1.0,77,69.56514,70.164837,70.662542,False,8,...,0,0,0,1,False,0,0,0,-16.9,6881.398125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,2022-08-31 19:00:00,22.5,0.9,84,71.22400,72.378973,72.792244,False,8,...,0,0,1,12,False,1,0,0,-34.5,2599.195439
16796,100,2022-08-31 20:00:00,20.7,0.4,95,68.95035,70.961363,71.899884,False,8,...,0,0,1,12,False,1,0,0,-34.4,2485.390799
16797,100,2022-08-31 21:00:00,20.2,0.4,98,68.24604,69.473463,70.866662,False,8,...,0,0,1,12,False,1,0,0,-35.3,2103.535017
16798,100,2022-08-31 22:00:00,20.1,1.1,97,68.01203,68.402807,69.828432,False,8,...,0,0,1,12,False,1,0,0,-36.8,2036.420278


In [109]:
print(sum(test_df['answer'].isna()))

0


In [190]:
test_df.columns

Index(['건물번호', '일시', '기온(C)', '풍속(m/s)', '습도(%)', '불쾌지수', '불쾌지수_이동평균3',
       '불쾌지수_이동평균5', 'holiday', '월', '일', '시간', '시간_sin', '시간_cos', '요일',
       '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', '건물기타', '공공', '대학교',
       '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소', '지식산업센터', '할인마트',
       '호텔및리조트', '건물유형_라벨', '태양광여부', 'work_time', 'low_day', 'particular',
       'CDH', 'answer'],
      dtype='object')

In [111]:
# 데이터 불러오기
df = pd.read_csv(path_drive+'sample_submission.csv')

df['answer'] = test_df['answer']
df.to_csv(path+'ensemble_final.csv', index=False)

In [112]:
len(df[df['answer']<0])

0